[lightgbmのinstall](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html#install-using-homebrew)

LightGBMを使うには"cmake"と"libomp"のインストールが必要

In [123]:
import lightgbm as lgb
import pandas as pd
import re
import time

In [124]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

pd.options.display.width = 500
pd.options.display.column_space = 200

In [125]:
results = pd.read_pickle('df.pickle')

# race_id_list = results.index.unique()
# race_id_list

In [126]:

# race_infos = {}
# # for race_id in tqdm(race_id_list):
# race_id = "202110040101"
# url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
# html = requests.get(url)
# html.encoding= "EUC_JP"
# soup = BeautifulSoup(html.text, "html.parser")

# test = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
# horse_id_list = []
# for a in test:
#   horse_id = re.findall(r'\d+',a["href"])
#   horse_id_list.append(horse_id[0])

In [127]:
# horse_id_list

In [128]:
race_id_lists = results.index.unique()[:13]

In [129]:
race_id_lists = ["202001010201"]

In [130]:
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      df = pd.read_html(url)[0]
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      # horse_id
      horse_id_list = []
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
        
      # print(df)
      print("horse_id_list")
      print(horse_id_list)
      print("jockey_id_list")
      print(jockey_id_list)
      print("df")
      print(df)
      df["horse_id"] = horse_id_list
      df["jockey_id"] = jockey_id_list
      
      race_results[_race_id] = df
      # print(race_results)
      
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_results

In [131]:
results_new = scrape_race_results(race_id_lists)
results_new

  0%|          | 0/1 [00:04<?, ?it/s]

horse_id_list
['2018110040', '2018105357', '2018100769', '2018105660', '2018105481', '2018106349', '2018103122']
jockey_id_list
['01170', '01032', '01140', '01176', '05339', '01025', '01095']
df
   着順  枠  馬番        馬名  性齢    斤量             騎手     タイム     着差  人気   単勝オッズ   後3F コーナー通過順     厩舎  馬体重(増減) Unnamed: 15 Unnamed: 16 Unnamed: 17
0   1  4   4    ボイオートス  牡2  54.0            横山武  1:10.2    NaN   3     4.0  35.3     4-3   美浦金成  480(-4)         NaN         NaN         NaN
1   1  7   7     ラピカズマ  牡2  54.0             池添  1:10.2     同着   1     2.9  35.7     1-1  栗東安田隆  464(+4)         NaN         NaN         NaN
2   3  3   3  デンプシーロール  牡2  54.0            横山和  1:10.2    アタマ   4     4.5  35.5     3-3   美浦黒岩  410(+2)         NaN         NaN         NaN
3   4  5   5    トーセンイヴ  牝2  52.0            △亀田  1:10.6  2.1/2   6    36.0  35.4     5-5   美浦小笠   422(0)         NaN         NaN         NaN
4   5  6   6  ワールドクルーズ  牝2  54.0  ルメード">マイスクワッド      牝2   54.0  勝浦  1:11.8   5.0       7   71.1     

ValueError: Length of values (7) does not match length of index (6)